# Modelo CNN

Antes de ser corrido este modelo devem ser criados os dados através do script de preprocessamento

### Imports

In [6]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from tensorflow.keras import models
from tensorflow.keras.layers import Dense
from tensorflow.python.keras import Input
from tensorflow.python.keras.layers import Conv1D, Flatten, MaxPooling1D, Dropout
import tensorflow as tf

## Carregamento dos dados

X é uma matriz de tamanho (max_sequence_len,100) 

Y é o EC Number (1-4 digitos)

In [2]:
data = pd.read_pickle("../parsed_data/data.pkl")

x = data['vectors'].tolist()
x = np.array(x, dtype='float32')


y = data['ec_number'].to_numpy()

## Conversão dos labels para One Hot Encoding

In [3]:
y = y.reshape(-1, 1)
enc = OneHotEncoder()
enc.fit(y)
y = enc.transform(y).toarray()

## Train test split

Separação dos dados em conjuntos de teste e treino.

In [4]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2)

## Criação do modelo

Este modelo foi criado com base no modelo do artigo "Identifying antimicrobial peptides using word embedding with deep recurrent neural networks" Faz uso de camadas Bidirecionais LSTM com valores intermédios de droput para resolver problemas de overfitting

Os testes neste modelo foram efetuados com os seguintes parametros de preprocessamento:

- maxlen = 700
- ecn_level = 1
- representation = 'matrix'

In [7]:
model = models.Sequential()
model.add(Input(shape=(698, 100), dtype='float32', name='main_input'))
model.add(Conv1D(64, 3))
model.add(MaxPooling1D(3, 2))
model.add(Dropout(0.1))
model.add(Conv1D(128, 3))
model.add(MaxPooling1D(3, 2))
model.add(Dropout(0.2))
model.add(Conv1D(128, 3))
model.add(MaxPooling1D(3, 2))
model.add(Dropout(0.3))
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(len(y[0]), activation='softmax'))
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['acc'])

model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d (Conv1D)              (None, 696, 64)           19264     
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 347, 64)           0         
_________________________________________________________________
dropout (Dropout)            (None, 347, 64)           0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 345, 128)          24704     
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 172, 128)          0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 172, 128)          0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 170, 128)         

## Treino do modelo

Treino do modelo construido com o conjuto de treino gerado.

In [ ]:
history = model.fit(x_train, y_train,
                    epochs=60,
                    batch_size=128,
                    validation_split=0.2)

## Validação

Validação do modelo usando o conjunto de dados de teste

In [ ]:
test_loss, test_acc = model.evaluate(x_test,  y_test, verbose=2)

print('\nTest accuracy:', test_acc)

## Detalhes do Treino do Modelo

Alguma informação sobre o treino do modelo em gráfico

In [ ]:
history_dict = history.history

acc_values = history_dict['acc']
val_acc_values = history_dict['val_acc']
epochs = range(1, len(acc_values) + 1)

plt.plot(epochs, acc_values, 'bo', label='Training acc')
plt.plot(epochs, val_acc_values, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Acc')
plt.legend()
plt.show()